In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends.dropna(inplace=True)

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.682900,3.831825,164.0,164.0,0.047729,12.023080,192.847214,13.886944,-0.799124,4.626670,164.0,164.0,0.017121,23.682823,759.846542,27.565314
0,86ba24037ffffff,1.048799,-6.371422,162.0,162.0,0.349978,6.978187,68.183222,8.257313,1.591410,-20.821766,162.0,162.0,0.422148,5.681497,115.697090,10.756258
0,86ba24067ffffff,0.427970,15.346140,209.0,209.0,0.009707,10.349081,414.217562,20.352336,0.478721,-14.865140,209.0,209.0,0.005541,22.421827,911.803854,30.196090
0,86ba2414fffffff,0.324461,5.646574,180.0,180.0,0.015013,6.597710,104.304301,10.212948,0.502958,-13.614059,180.0,180.0,0.051954,7.333325,69.708509,8.349162
0,86ba2415fffffff,0.282171,2.481903,290.0,290.0,0.002886,15.072211,749.202727,27.371568,-0.393635,-9.193683,290.0,290.0,0.002900,27.708999,1450.815964,38.089578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3707ffffff,0.865004,22.269126,225.0,225.0,0.079626,19.285233,499.148611,22.341634,0.195330,13.494881,225.0,225.0,0.082748,3.475579,24.409402,4.940587
0,86dab370fffffff,0.290373,27.583054,205.0,205.0,0.015507,15.783516,321.270519,17.924021,-0.220059,20.163691,205.0,205.0,0.007306,14.071795,394.893103,19.871917
0,86dab3737ffffff,1.944379,-13.752724,167.0,167.0,0.635050,7.921065,124.048345,11.137699,0.066460,-1.138890,167.0,167.0,0.043398,0.976431,5.558817,2.357714
0,86dab3757ffffff,-0.721749,22.942676,196.0,196.0,0.083120,15.310604,330.746441,18.186436,0.312192,10.113820,196.0,196.0,0.062100,7.683109,84.727679,9.204764


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    
# cell_trends['hex_id'].to_csv("global-inputs/valid_cells.csv")

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.285957,-4.315665,131.0,131.0,0.026964,...,106.866935,10.337646,1.110527,-20.523659,131.0,131.0,0.048771,22.488229,871.112306,29.514612
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.520916,2.070863,79.0,79.0,0.087756,...,59.301465,7.700744,0.461562,-4.387515,79.0,79.0,0.038324,9.828792,112.387894,10.601316
2,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.826371,-0.681890,7.0,7.0,0.081622,...,29.049672,5.389775,3.016152,8.967772,7.0,7.0,0.070048,18.404964,456.612787,21.368500
3,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.243037,24.747216,146.0,146.0,0.002179,...,646.359363,25.423599,1.891280,-59.854557,146.0,146.0,0.053528,34.193643,1511.580678,38.879052
4,86bb2bae7ffffff,86bb2bae7ffffff,85bb2baffffffff,44.20,"POLYGON ((175.14401 -40.80153, 175.13871 -40.7...",1.254935,4.223656,90.0,90.0,0.161833,...,433.334880,20.816697,1.015927,-9.716242,90.0,90.0,0.183530,10.307552,243.936212,15.618457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,86da9285fffffff,86da9285fffffff,85da9287fffffff,2.00,"POLYGON ((171.98283 -41.59482, 171.97888 -41.5...",-0.265652,6.143397,249.0,249.0,0.010138,...,428.668293,20.704306,0.083607,10.767854,249.0,249.0,0.073099,1.320281,5.513995,2.348190
631,86da94ab7ffffff,86da94ab7ffffff,85da94abfffffff,50.36,"POLYGON ((172.82222 -43.2102, 172.81779 -43.17...",-0.173030,-10.364882,170.0,170.0,0.005706,...,345.740963,18.594111,0.307729,-2.451333,170.0,170.0,0.181706,4.280967,28.263547,5.316347
632,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.808723,12.175997,234.0,234.0,0.062074,...,436.229195,20.886101,0.093501,-0.353060,234.0,234.0,0.300368,0.796162,0.898887,0.948096
633,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.879839,-40.998040,310.0,310.0,0.291147,...,702.427260,26.503344,-4.574011,22.565704,310.0,310.0,0.191088,43.181769,3080.933343,55.506156
